In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
pwd()

"/Users/ivanspirandelli/Doktor/Code/MorphoMol/MorphoMolNotebooks"

In [4]:
folder = "../../Data/hpc_out/rwm_op_3_6r7m/"
mol_type = "6r7m"
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"

id_index = 1
separator = "."
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println("Error in $file")
        end
    end
end

In [5]:
argmin(min_Es)

43

In [32]:
for elem in evaluation_strings
    println(elem)
end


2: 2.0 | 0.50442475 | 45.96227452649346, -110.94548415517777, -109.8287898383199 | 16.16857392224261
3: 2.0 | 0.299293 | 90.42696720224309, -117.75077790862684, -108.00833203467644 | 40.86532880846855
4: 2.0 | 0.36299214 | 1.0880041047117572, -108.51988347109341, -100.59293725342182 | 31.352217757108065
5: 2.0 | 0.23264985 | 76.67544966942425, -165.50762468042507, -158.62115128330353 | 53.77609737793702
6: 2.0 | 0.567495 | -64.69230786915028, -94.45214622629898, -87.79091737346744 | 31.85920417612689
7: 2.0 | 0.3290469 | 18.20958033804439, -129.39199867189694, -122.40315502260262 | 55.14244722061476
8: 2.0 | 0.3261218 | 102.7085417379931, -103.85252208710801, -93.26682388043335 | 51.05792117092392
9: 2.0 | 0.16321656 | 9.161727906758443, -163.3047489863447, -159.5602436801995 | 67.78514273677933
10: 2.0 | 0.275 | -0.8248326073289718, -164.5327388360667, -163.3835897573619 | 64.80966646312964


In [33]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")
JLD2.InvalidDataException("Did not find a Superblock.")


# Print Single Assembly Path

In [34]:
@load "../../Data/hpc_out/rwm_op_3_6r7m/5_rwm_op_3_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [35]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [36]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
